In [7]:
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from models.py import GCN
import importlib
import utils
importlib.reload(utils)
from utils import *

# Set random seed
seed = 123
np.random.seed(seed)


ModuleNotFoundError: No module named 'inits.py'; 'inits' is not a package

In [2]:
# get paths to available grahs metadata
N_sample = 200000
adj_mat, features_mat, train_labels_mat, test_labels_mat, val_labels_mat, train_mask_mat, test_mask_mat, val_mask_mat = load_data("txt_graph2216_21012020", N_sample)

In [3]:
print(adj_mat.shape)
print(features_mat.shape)
print(train_labels_mat.shape)
print(test_labels_mat.shape)
print(val_labels_mat.shape)
print(train_mask_mat.shape)
print(test_mask_mat.shape)
print(val_mask_mat.shape)

print("number of masked elements")
print("train: " + str(len(np.where(train_mask_mat[:,1]==1)[0])))
print("test: " + str(len(np.where(test_mask_mat[:,1]==1)[0])))
print("val: " + str(len(np.where(val_mask_mat[:,1]==1)[0])))

print("\n positions of ones in masked labels")
print(np.where(train_labels_mat[np.array(train_mask_mat[:,1],dtype=bool)]==1)[1])
print(np.where(test_labels_mat[np.array(test_mask_mat[:,1],dtype=bool)]==1)[1])
print(np.where(val_labels_mat[np.array(val_mask_mat[:,1],dtype=bool)]==1)[1])

print("\n ones outside the mask")
print(np.where(train_labels_mat[np.logical_not(np.array(train_mask_mat[:,1],dtype=bool))]==1))
print(np.where(test_labels_mat[np.logical_not(np.array(test_mask_mat[:,1],dtype=bool))]==1))
print(np.where(val_labels_mat[np.logical_not(np.array(val_mask_mat[:,1],dtype=bool))]==1))

#sum_global_nodes = np.sum(adj_mat.toarray()[:,np.array(train_mask_mat[:,1],dtype=bool)], axis = 1) + np.sum(adj_mat.toarray()[:,np.array(test_mask_mat[:,1],dtype=bool)], axis = 1) + np.sum(adj_mat.toarray()[:,np.array(val_mask_mat[:,1],dtype=bool)], axis = 1)
#print(adj_mat.shape[1] - sum(sum_global_nodes))

(465542, 465542)
(465542, 100)
(465542, 5)
(465542, 5)
(465542, 5)
(465542, 5)
(465542, 5)
(465542, 5)
number of masked elements
train: 779
test: 668
val: 778

 positions of ones in masked labels
[2 1 0 4 0 4 4 1 3 0 3 3 0 4 2 0 0 1 4 1 1 3 0 2 0 1 1 0 4 0 4 3 3 2 2 0 2
 2 4 4 1 3 2 2 0 4 4 4 3 1 2 0 3 1 2 0 1 3 4 0 3 3 3 4 0 1 4 1 3 4 3 1 1 1
 3 0 0 4 3 2 1 0 0 0 2 0 4 3 0 0 4 4 3 2 3 3 1 4 3 4 1 0 3 2 4 3 4 4 0 1 4
 3 1 0 1 3 3 4 4 3 3 3 4 3 2 3 1 2 3 1 3 0 2 0 3 2 4 2 0 2 0 4 2 0 2 1 0 2
 0 0 3 0 1 2 3 3 3 1 4 3 2 4 3 3 0 4 3 1 3 1 3 0 4 2 4 2 0 3 2 3 2 2 2 0 2
 2 4 0 2 3 0 2 0 0 0 2 0 0 2 2 2 3 2 0 3 1 4 3 0 0 4 1 2 2 4 0 2 3 0 4 2 3
 0 0 2 1 0 0 3 3 3 1 2 4 0 3 0 3 3 1 4 3 2 4 2 3 4 0 3 4 0 3 3 4 0 0 0 3 3
 0 2 1 4 4 3 0 0 4 4 1 1 4 3 0 3 4 3 2 3 2 1 4 3 4 0 3 0 4 4 0 4 1 2 2 3 3
 3 0 2 4 3 3 2 3 4 2 1 0 2 4 4 1 4 3 4 3 1 3 3 1 3 2 0 4 1 0 2 3 0 4 4 4 1
 3 3 4 1 0 4 2 3 3 0 0 1 2 3 1 0 4 0 2 3 1 1 0 2 3 3 1 0 4 3 3 2 4 0 0 1 1
 0 0 0 2 0 0 3 3 4 3 1 1 0 4 2 1 0 0 3 1 4 1 3 0 3 1 2

In [15]:
#delete all flags before declaration new one
del_all_flags(tf.flags.FLAGS)

# Settings
flags = tf.app.flags
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('mode', '', 'kernel') # for line by line mode
tf.app.flags.DEFINE_string('port', '', 'kernel') # for line by line mode
tf.app.flags.DEFINE_string('f', '', 'kernel') # for jupyter notebook

flags.DEFINE_float('learning_rate', 0.03, 'Initial learning rate.') #0.01
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.') #200
flags.DEFINE_integer('hidden1', 64, 'Number of units in hidden layer 1.') #16
flags.DEFINE_float('dropout', 0.6, 'Dropout rate (1 - keep probability).') #0.5
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 20, 'Tolerance for early stopping (# of epochs).') #10

# Some preprocessing
features = preprocess_features(features_mat)
support = [preprocess_adj(adj_mat)]
num_supports = 1
model_func = GCN 

/home/robertb/python/GCN_text_classif/gcn/utils.py:57: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


In [5]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, train_labels_mat.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
# Initialize session
sess = tf.Session()
# Init variables
sess.run(tf.global_variables_initializer())

In [7]:
# Train model
cost_val = []

for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, train_labels_mat, train_mask_mat[:,1], placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, val_labels_mat, val_mask_mat[:,1], placeholders, sess, model)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

Epoch: 0001 train_loss= 30.33150 train_acc= 0.22721 val_loss= 39.41757 val_acc= 0.72237 time= 10.71388
Epoch: 0002 train_loss= 16.81936 train_acc= 0.20924 val_loss= 37.68326 val_acc= 0.70823 time= 10.01661
Epoch: 0003 train_loss= 26.68737 train_acc= 0.22593 val_loss= 36.66272 val_acc= 0.69280 time= 10.01657
Epoch: 0004 train_loss= 28.54598 train_acc= 0.21823 val_loss= 35.95023 val_acc= 0.66838 time= 10.07508
Epoch: 0005 train_loss= 20.57083 train_acc= 0.22721 val_loss= 35.04095 val_acc= 0.64910 time= 10.03505
Epoch: 0006 train_loss= 23.62959 train_acc= 0.21566 val_loss= 34.01050 val_acc= 0.62982 time= 10.04319
Epoch: 0007 train_loss= 25.53231 train_acc= 0.19384 val_loss= 32.79931 val_acc= 0.62725 time= 10.14809
Epoch: 0008 train_loss= 23.18822 train_acc= 0.23363 val_loss= 31.68569 val_acc= 0.61954 time= 9.98431
Epoch: 0009 train_loss= 21.49719 train_acc= 0.21053 val_loss= 30.61210 val_acc= 0.61440 time= 10.07334
Epoch: 0010 train_loss= 33.64248 train_acc= 0.19384 val_loss= 29.52277 val

Epoch: 0081 train_loss= 19.02888 train_acc= 0.22080 val_loss= 30.96159 val_acc= 0.34190 time= 10.09306
Epoch: 0082 train_loss= 20.47910 train_acc= 0.22721 val_loss= 30.85707 val_acc= 0.34319 time= 10.16634
Epoch: 0083 train_loss= 12.13223 train_acc= 0.21823 val_loss= 30.75383 val_acc= 0.34190 time= 10.56029
Epoch: 0084 train_loss= 19.33172 train_acc= 0.20924 val_loss= 30.67305 val_acc= 0.34319 time= 10.25206
Epoch: 0085 train_loss= 13.63630 train_acc= 0.19255 val_loss= 30.61614 val_acc= 0.34319 time= 10.09942
Epoch: 0086 train_loss= 18.40340 train_acc= 0.21053 val_loss= 30.58290 val_acc= 0.34319 time= 10.02798
Epoch: 0087 train_loss= 16.76731 train_acc= 0.18870 val_loss= 30.56355 val_acc= 0.34190 time= 10.00977
Epoch: 0088 train_loss= 15.22830 train_acc= 0.22465 val_loss= 30.48252 val_acc= 0.33933 time= 10.10460
Epoch: 0089 train_loss= 21.75092 train_acc= 0.21053 val_loss= 30.35401 val_acc= 0.33548 time= 10.02415
Epoch: 0090 train_loss= 16.74750 train_acc= 0.21823 val_loss= 30.27246 va

Epoch: 0161 train_loss= 10.67792 train_acc= 0.24647 val_loss= 21.90744 val_acc= 0.33162 time= 10.01205
Epoch: 0162 train_loss= 15.27615 train_acc= 0.24005 val_loss= 21.80357 val_acc= 0.33548 time= 10.04854
Epoch: 0163 train_loss= 11.57002 train_acc= 0.20924 val_loss= 21.73407 val_acc= 0.33290 time= 10.13722
Epoch: 0164 train_loss= 9.30929 train_acc= 0.24647 val_loss= 21.67608 val_acc= 0.33033 time= 10.09720
Epoch: 0165 train_loss= 15.22788 train_acc= 0.22721 val_loss= 21.63949 val_acc= 0.32391 time= 10.04852
Epoch: 0166 train_loss= 13.88878 train_acc= 0.21053 val_loss= 21.59272 val_acc= 0.32134 time= 10.00591
Epoch: 0167 train_loss= 8.65047 train_acc= 0.22080 val_loss= 21.55916 val_acc= 0.32262 time= 10.02684
Epoch: 0168 train_loss= 17.35087 train_acc= 0.21694 val_loss= 21.51568 val_acc= 0.32391 time= 9.99444
Epoch: 0169 train_loss= 11.91391 train_acc= 0.21181 val_loss= 21.51903 val_acc= 0.32519 time= 10.26238
Epoch: 0170 train_loss= 10.18093 train_acc= 0.24775 val_loss= 21.51010 val_a

Epoch: 0241 train_loss= 9.41452 train_acc= 0.23363 val_loss= 17.50740 val_acc= 0.38303 time= 10.10352
Epoch: 0242 train_loss= 7.88681 train_acc= 0.24775 val_loss= 17.48282 val_acc= 0.38303 time= 10.10715
Epoch: 0243 train_loss= 9.22976 train_acc= 0.21566 val_loss= 17.43946 val_acc= 0.38432 time= 10.08175
Epoch: 0244 train_loss= 9.06951 train_acc= 0.23620 val_loss= 17.41250 val_acc= 0.38560 time= 10.11469
Epoch: 0245 train_loss= 8.34277 train_acc= 0.21309 val_loss= 17.38198 val_acc= 0.38560 time= 10.01817
Epoch: 0246 train_loss= 10.15687 train_acc= 0.24519 val_loss= 17.37591 val_acc= 0.39075 time= 10.01461
Epoch: 0247 train_loss= 7.10354 train_acc= 0.22978 val_loss= 17.37362 val_acc= 0.39075 time= 10.08671
Epoch: 0248 train_loss= 9.65806 train_acc= 0.25802 val_loss= 17.35454 val_acc= 0.39203 time= 10.04643
Epoch: 0249 train_loss= 5.31145 train_acc= 0.23235 val_loss= 17.33066 val_acc= 0.39075 time= 10.02977
Epoch: 0250 train_loss= 7.19428 train_acc= 0.26829 val_loss= 17.30233 val_acc= 0.

Epoch: 0322 train_loss= 6.17212 train_acc= 0.21951 val_loss= 13.49052 val_acc= 0.32905 time= 11.03098
Epoch: 0323 train_loss= 7.59732 train_acc= 0.22080 val_loss= 13.46160 val_acc= 0.33033 time= 10.10971
Epoch: 0324 train_loss= 5.29685 train_acc= 0.25417 val_loss= 13.43024 val_acc= 0.32905 time= 10.03883
Epoch: 0325 train_loss= 6.13692 train_acc= 0.26059 val_loss= 13.39267 val_acc= 0.33162 time= 10.15017
Epoch: 0326 train_loss= 8.13469 train_acc= 0.23235 val_loss= 13.37054 val_acc= 0.33290 time= 10.13595
Epoch: 0327 train_loss= 6.43751 train_acc= 0.22208 val_loss= 13.34554 val_acc= 0.33290 time= 10.05798
Epoch: 0328 train_loss= 6.70601 train_acc= 0.23363 val_loss= 13.32617 val_acc= 0.33162 time= 10.02800
Epoch: 0329 train_loss= 7.49256 train_acc= 0.22850 val_loss= 13.30678 val_acc= 0.33290 time= 10.05106
Epoch: 0330 train_loss= 7.13865 train_acc= 0.24134 val_loss= 13.27764 val_acc= 0.33290 time= 10.10111
Epoch: 0331 train_loss= 5.86257 train_acc= 0.21181 val_loss= 13.25172 val_acc= 0.3

...

Epoch: 1631 train_loss= 1.66819 train_acc= 0.23620 val_loss= 8.08911 val_acc= 0.95244 time= 12.02154
Epoch: 1632 train_loss= 1.66716 train_acc= 0.25289 val_loss= 8.08840 val_acc= 0.95116 time= 12.05427
Epoch: 1633 train_loss= 1.71769 train_acc= 0.24390 val_loss= 8.08765 val_acc= 0.94602 time= 11.97112
Epoch: 1634 train_loss= 1.68909 train_acc= 0.24134 val_loss= 8.08713 val_acc= 0.94473 time= 11.92887
Epoch: 1635 train_loss= 1.68757 train_acc= 0.24134 val_loss= 8.08638 val_acc= 0.94216 time= 12.03521
Epoch: 1636 train_loss= 1.66604 train_acc= 0.24904 val_loss= 8.08575 val_acc= 0.92288 time= 11.97456
Epoch: 1637 train_loss= 1.71101 train_acc= 0.22978 val_loss= 8.08515 val_acc= 0.89589 time= 11.96119
Epoch: 1638 train_loss= 1.72270 train_acc= 0.23107 val_loss= 8.08399 val_acc= 0.88946 time= 12.68744
Epoch: 1639 train_loss= 1.66719 train_acc= 0.24904 val_loss= 8.08275 val_acc= 0.88175 time= 11.23083
Epoch: 1640 train_loss= 1.65936 train_acc= 0.25160 val_loss= 8.08204 val_acc= 0.86761 time=

Epoch: 1713 train_loss= 1.69234 train_acc= 0.21694 val_loss= 8.08352 val_acc= 0.90360 time= 12.02432
Epoch: 1714 train_loss= 1.67376 train_acc= 0.24005 val_loss= 8.08243 val_acc= 0.90488 time= 12.86000
Epoch: 1715 train_loss= 1.67313 train_acc= 0.23620 val_loss= 8.08200 val_acc= 0.90360 time= 11.94961
Epoch: 1716 train_loss= 1.67075 train_acc= 0.26059 val_loss= 8.08173 val_acc= 0.88046 time= 12.00440
Epoch: 1717 train_loss= 1.64150 train_acc= 0.24775 val_loss= 8.08149 val_acc= 0.82262 time= 12.01163
Epoch: 1718 train_loss= 1.64067 train_acc= 0.26316 val_loss= 8.08095 val_acc= 0.75064 time= 11.31630
Epoch: 1719 train_loss= 1.65097 train_acc= 0.25674 val_loss= 8.08117 val_acc= 0.66581 time= 10.31072
Epoch: 1720 train_loss= 1.65382 train_acc= 0.25160 val_loss= 8.08172 val_acc= 0.72879 time= 10.26047
Epoch: 1721 train_loss= 1.65845 train_acc= 0.25032 val_loss= 8.08309 val_acc= 0.82905 time= 10.23085
Epoch: 1722 train_loss= 1.66038 train_acc= 0.25674 val_loss= 8.08401 val_acc= 0.89460 time=

Epoch: 1795 train_loss= 1.66230 train_acc= 0.23748 val_loss= 8.08098 val_acc= 0.95758 time= 10.23281
Epoch: 1796 train_loss= 1.66905 train_acc= 0.25802 val_loss= 8.08093 val_acc= 0.94987 time= 10.31614
Epoch: 1797 train_loss= 1.67050 train_acc= 0.27086 val_loss= 8.08192 val_acc= 0.93188 time= 10.22674
Epoch: 1798 train_loss= 1.67545 train_acc= 0.25160 val_loss= 8.08376 val_acc= 0.89460 time= 10.22856
Epoch: 1799 train_loss= 1.64781 train_acc= 0.23620 val_loss= 8.08577 val_acc= 0.84704 time= 10.20597
Epoch: 1800 train_loss= 1.68771 train_acc= 0.25802 val_loss= 8.08739 val_acc= 0.80720 time= 10.24014
Epoch: 1801 train_loss= 1.67247 train_acc= 0.25417 val_loss= 8.08864 val_acc= 0.76864 time= 10.21636
Epoch: 1802 train_loss= 1.61493 train_acc= 0.26573 val_loss= 8.08984 val_acc= 0.76350 time= 10.27442
Epoch: 1803 train_loss= 1.69501 train_acc= 0.24519 val_loss= 8.09063 val_acc= 0.76350 time= 10.27950
Epoch: 1804 train_loss= 1.67585 train_acc= 0.26573 val_loss= 8.09114 val_acc= 0.82648 time=

Epoch: 1877 train_loss= 1.64832 train_acc= 0.23748 val_loss= 8.08618 val_acc= 0.85090 time= 10.20983
Epoch: 1878 train_loss= 1.67846 train_acc= 0.25546 val_loss= 8.08720 val_acc= 0.88175 time= 10.21126
Epoch: 1879 train_loss= 1.68599 train_acc= 0.24005 val_loss= 8.08819 val_acc= 0.89589 time= 10.21837
Epoch: 1880 train_loss= 1.63428 train_acc= 0.24904 val_loss= 8.08893 val_acc= 0.91131 time= 10.21575
Epoch: 1881 train_loss= 1.68094 train_acc= 0.24134 val_loss= 8.08896 val_acc= 0.92288 time= 10.74168
Epoch: 1882 train_loss= 1.65454 train_acc= 0.24904 val_loss= 8.08872 val_acc= 0.94987 time= 10.36053
Epoch: 1883 train_loss= 1.70166 train_acc= 0.23748 val_loss= 8.08793 val_acc= 0.95630 time= 10.23291
Epoch: 1884 train_loss= 1.67150 train_acc= 0.24262 val_loss= 8.08735 val_acc= 0.95758 time= 10.22510
Epoch: 1885 train_loss= 1.63701 train_acc= 0.25032 val_loss= 8.08676 val_acc= 0.94859 time= 10.22137
Epoch: 1886 train_loss= 1.65403 train_acc= 0.23235 val_loss= 8.08607 val_acc= 0.92674 time=

Epoch: 1959 train_loss= 1.66035 train_acc= 0.24775 val_loss= 8.08663 val_acc= 0.76735 time= 10.23044
Epoch: 1960 train_loss= 1.63919 train_acc= 0.24519 val_loss= 8.08555 val_acc= 0.68509 time= 10.34826
Epoch: 1961 train_loss= 1.66013 train_acc= 0.25674 val_loss= 8.08439 val_acc= 0.60411 time= 10.25741
Epoch: 1962 train_loss= 1.64671 train_acc= 0.23620 val_loss= 8.08336 val_acc= 0.47044 time= 10.27655
Epoch: 1963 train_loss= 1.63487 train_acc= 0.26316 val_loss= 8.08280 val_acc= 0.44602 time= 10.23689
Epoch: 1964 train_loss= 1.64557 train_acc= 0.27985 val_loss= 8.08349 val_acc= 0.52956 time= 10.21841
Epoch: 1965 train_loss= 1.65360 train_acc= 0.24775 val_loss= 8.08400 val_acc= 0.60797 time= 10.20750
Epoch: 1966 train_loss= 1.66511 train_acc= 0.24775 val_loss= 8.08447 val_acc= 0.73136 time= 10.22461
Epoch: 1967 train_loss= 1.64808 train_acc= 0.26316 val_loss= 8.08462 val_acc= 0.75450 time= 10.23000
Epoch: 1968 train_loss= 1.68261 train_acc= 0.26958 val_loss= 8.08464 val_acc= 0.78406 time=

In [12]:
# Testing
test_cost, test_acc, test_duration = evaluate(features, support, val_labels_mat, val_mask_mat[:,1], placeholders, sess, model)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))

Test set results: cost= 1.61192 accuracy= 0.27121 time= 6.48976
[39.417572, 37.68326, 36.662716, 35.950226, 35.040947, 34.0105, 32.79931, 31.685692, 30.612103, 29.522766, 28.421473, 27.267614, 26.04314, 24.885725, 23.816912, 22.911482, 23.14679, 23.502064, 23.898617, 24.375204, 24.817476, 25.239853, 25.526617, 25.858856, 26.139387, 26.424393, 26.627184, 26.812918, 27.091627, 27.36971, 27.593872, 27.979736, 28.283022, 28.618864, 29.029453, 29.448526, 29.876125, 30.279856, 30.6507, 30.958248, 31.245161, 31.501953, 31.7274, 31.950003, 32.24564, 32.449863, 32.61364, 32.748634, 32.787884, 32.878876, 32.916332, 32.883392, 32.85091, 32.859634, 32.95164, 32.96809, 32.98352, 33.06282, 33.126617, 33.15312, 33.240543, 33.33949, 33.372524, 33.442852, 33.45361, 33.35721, 33.25648, 33.16203, 33.048542, 32.84707, 32.676517, 32.422215, 32.205143, 31.94729, 31.657776, 31.421373, 31.261932, 31.153067, 31.088106, 31.04863, 30.961594, 30.857067, 30.753826, 30.67305, 30.616137, 30.582897, 30.563549, 30.482